# SIT742: Modern Data Science 
**(Assessment Task 01: Wine Rating Data Exploration)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, change and distribute this package.

Prepared by **SIT742 Teaching Team**


---

**Student Information:** Please fill your information below

- Name: Pooja Bhat
- Student ID: 218545396
- Email: pabhat@deakin.edu.au

---

# Part 0. Data Files

## 0.1 Download Data  

In [0]:
!pip install wget  

In [0]:
import wget

link_to_data = 'https://github.com/tulip-lab/sit742/raw/master/Assessment/2019/data/wine.json'
DataSet = wget.download(link_to_data)

link_to_data = 'https://github.com/tulip-lab/sit742/raw/master/Assessment/2019/data/stopwords.txt'

DataSet = wget.download(link_to_data)

In [0]:
!ls

## 0.2 Load Data  

In [0]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
file = 'wine.json'

In [0]:
# write your code here
# read the json file and drop rows with invalid values in the attributes of 'points' and 'price'.
winedf_1 = pd.read_json("wine.json")

# check if there are nulls in price and points attributes
print(winedf_1.isnull().any())
# output shows that price attribute has nulls, but point attribute doesnt have any nulls

#drop nulls from price attribute
winedf_1.dropna(subset=['price'],inplace=True)

# confirm that null rows are dropped from price attribute
print('-------------------------------------')
print('With Nulls dropped in price attribute')
print('-------------------------------------')
print(winedf_1.isnull().any())

# Part 1: numeric anaysis

## 1.1 Explore the data distribution for each column.

In [0]:
# write your code here
# you may use functions such as describe() on each attribute

# Check the datatypes of each of the attributes
print('-------------------------------------')
print('Understanding datatypes of attributes')
print('-------------------------------------')
print(winedf_1.info())
print('Output shows that only Price and points are numeric attributes')
print('\n')

print('-------------------------------------')
print('Ratings by coutries, ')
print('-------------------------------------')
print(winedf_1['country'].value_counts())
print('Output shows that maximum number of ratings are from US, follwed by France and Italy')
print('\n')

print('-------------------------------------')
print('Reviewers counts')
print('-------------------------------------')
print(winedf_1['taster_twitter_handle'].value_counts())
print('Output shows @vossroger has the highest number of reviews followed by @wineschach')
print('\n')

print('--------------------------------------------')
print('Overall data distribution summary statistics')
print('--------------------------------------------')
winedf_1.describe()

**Observations from the Data Distribution**

1) Price and Points are numeric attributes, all other attributes are categorical attributes 

2) Maximum number of ratings are from US, follwed by France and Italy

3) Reviewer @vossroger has the highest number of reviews followed by @wineschach

4) The Summary Statistic for Points shows that the maximum points awarded to wines were 100 and minimum were 80 with a mean of 88.42 and a standard deviation of 3.044508. The middle 50% of the data was between 86 and 91 points.

4) The Summary Statistic for Price shows that the maximum priceof wine was 3,300 dollars and minimum was 4 dollars with a mean of 35.36 dollars and a standard deviation of 41.02 dollars. The middle 50% of the price was between 17 and 42 dollars.

## 1.2 Find the 10 varieties of wine which receives the highest number of  reviews

In [0]:
# write your code here
# you may use functions such as value_counts()  
print('--------------------------------------------')
print('10 Wine Varieties with highest reviews')
print('--------------------------------------------')
winedf_1['variety'].value_counts().head(10)

## 1.3 Find varieties of wine having the average price less than 20, with the average points at least 90

In [0]:
# write your code here
# you may use functions such as groupby() 
print('----------------------------------------------------------------------------')
print('Wine Varieties with average price less than 20 and average points atleast 90')
print('----------------------------------------------------------------------------')
grouped_variety=winedf_1.groupby('variety')[['price','points']].mean()
grouped_variety[(grouped_variety.price < 20) & (grouped_variety.points>=90)]

## 1.3 Build statistic table

In [0]:
# write your code here
# you may use functions such as groupby() and round(decimals=2)
# First we will group by country's to find the variety of wines that have the higest reviews
grouped_country_1=winedf_1.groupby('country').apply(lambda x: x['variety'].value_counts() == x['variety'].value_counts().max()).reset_index(name='variety')
grouped_country_variety=grouped_country_1.loc[grouped_country_1['variety'] == True].drop(columns=['variety'])
grouped_country_variety=grouped_country_variety.rename(columns={"level_1": "variety"})
#print(grouped_country_variety)
grouped_country_variety=grouped_country_variety.groupby('country', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
#print(grouped_country_variety)

# Next we will group by country to find the average of price and points for that country 
grouped_country_2=winedf_1.groupby(['country'])[['points','price']].mean().round(decimals=2)
grouped_country_2=grouped_country_2.reset_index()
#print(grouped_country_2)

# We will merge the 2 dataframes to create a single dataframe, that will help us with deciding the best wine for recmmending hotel Tulip
result_df = pd.merge(grouped_country_variety, grouped_country_2,
                 on='country')

# Sort result by points
#print("Sort result by Points")
#print(result_df.sort_values(by=['points'],ascending=False))

# Sort result by price
#print("Sort result by Price")
#print(result_df.sort_values(by=['price'],ascending=False))

result_df.columns = ['Country', 'Variety','AvgPoint','AvgPrice']
result_df.sort_values(by=['AvgPoint'],ascending=False)



**Recommendations to Hotel *TULIP* **

Based on this analysis, I will advise hotel *Tulip* to order from India since the avergage points are the second higest and the average price is very reasonable at $13.33

Next, I would also recommend ordering from China, since the reviewers gave it a high rating of 89 points and the price was reasonable at $18.

Although, the above analysis on the ratings by country shows that India only had 9 reviews and Chine only had 1 review. 

I would also encourage Hotel Tulip to have some fine wines from countries with higher number of reviews such as Austria and Germany which have considerably high avergage ratings of 90.19 and 89.84 and the prices are between 30.76 and 42.26 respetively



In [0]:
# save your table to 'statisticByState.csv'
result_df.to_csv(r'statisticByState.csv',index=False,encoding="utf-8-sig")

# Part 2. Text Analysis

## 2.1 extract high requency words in description

In [0]:
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.probability import *
from itertools import chain
#from tqdm import tqdm
import codecs

In [0]:
with open('stopwords.txt') as f:
    stop_words = f.read().splitlines()
stop_words = set(stop_words)

In [0]:
# write your code here
# define your tokenize
winedf_tokenized=winedf_1.copy()
tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)?") 
winedf_tokenized['description']=winedf_tokenized['description'].apply(tokenizer.tokenize)
#winedf_tokenized['description']=winedf_tokenized['description'].apply(lambda x: [item.lower() for item in x if item.lower() not in stop_words and item.isalpha()])
winedf_tokenized['description']=winedf_tokenized['description'].apply(lambda x: [item.lower() for item in x if item.lower() not in stop_words])
winedf_tokenized.head(10)

In [0]:
# find top common words with document frequencies > 5000
# you may use function FreqDist() and sort()
from __future__ import division
from itertools import chain

winedf_tokenized_freq=winedf_tokenized['description'].copy()
words = list(chain.from_iterable(winedf_tokenized_freq))
Frequent_words=FreqDist(words)

#Most common 100 words
#print(Frequent_words.most_common(100))

MostFreqWords = set([k for k, v in Frequent_words.items() if v >5000])
MostFreqWords

In [0]:
# save your table to 'top_common_words.txt'  
with open('HighFreq.txt', 'w') as f:
    for item in sorted(MostFreqWords):
        f.write("%s\n" % item)
        print(item)

## 2.2 Find key words for describing Shiraz using TF-IDF

In [0]:
# select 'description' from 'variety' equal to  'Shiraz' 
winedf_Shiraz=winedf_tokenized[winedf_tokenized['variety']=="Shiraz"]
winedf_Shiraz_2=winedf_Shiraz.copy()
winedf_Shiraz_2

In [0]:
# use TfidfVectorizer to calculate TF-IDF score
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='word', stop_words = stop_words) 

winedf_Shiraz_2['description_1']=winedf_Shiraz_2['description'].apply(lambda x: ' '.join(map(str,x)))
tfs = tfidf.fit_transform(winedf_Shiraz_2['description_1'] )
tfs.shape
#print(winedf_Shiraz_2['description_1'])

In [0]:
# find words with TF-IDF score >0.4 and sort them
my_list = []
vocab = tfidf.get_feature_names()
for i in range(tfs.shape[0]):
    for word, weight in zip(vocab, tfs.toarray()[i]):
        if weight > 0.4:
            if word not in my_list:
                print (word, ":", weight)
                my_list.append(word)
            
print(my_list)      


In [0]:
# save your table to 'key_Shiraz.txt'   
with open('Shirazkey.txt', 'w') as f:
    for item in sorted(my_list):
        f.write("%s\n" % item)
        print(item)